This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd # standard naming convention
import numpy as np
from random import randint

data = pd.read_csv('data/data_000637.txt')
df = pd.DataFrame(data)
N = randint(10000, 1310720)
df.head(N)
#print('Number of rows printed: ', N)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1251939,1,1,2,3869210773,54,8
1251940,1,0,60,3869210773,58,17
1251941,1,0,64,3869210773,54,11
1251942,1,0,41,3869210773,53,11


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [3]:
df_cntbx = df.loc[:, ['ORBIT_CNT', 'BX_COUNTER']]
df_cnt = df.loc[:, ['ORBIT_CNT']]
df_bx = df.loc[:, ['BX_COUNTER']]
'''
save_position = 0
save_number_bxcounter = 0
n_orbits = 0
nrows = len(df)
for i in range(nrows):
    orbit_cnt_int1 = int(df_cnt.iloc[i])
    orbit_cnt_int2 = int(df_cnt.iloc[i+1])
    if orbit_cnt_int1 != orbit_cnt_int2:
        orbit_bx_last = int(df_bx.iloc[i])
        orbit_bx_first = int(df_bx.iloc[save_position])
        sub = orbit_bx_last-orbit_bx_first
        save_number_bxcounter += sub
        n_orbits += 1
        #print('ORBIT_CNT value:', orbit_cnt_int1, '\nBX_COUNTER estimation:', sub, '\n')
        save_position = i+1
print('BX_COUNTER estimation in 1 orbit:', float(save_number_bxcounter/n_orbits))
'''

df_cntbx = df_cntbx.groupby(['ORBIT_CNT'], as_index=False)['BX_COUNTER'].sum()
df_cntbx.head()

,ORBIT_CNT,BX_COUNTER
0,3869200167,124133
1,3869200168,97201
2,3869200169,144343
3,3869200170,217462
4,3869200171,228822


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [4]:
df_maxTDC = df.groupby(['TDC_MEAS'], as_index=False).max()
print(df_maxTDC.iloc[-3:])



    TDC_MEAS  HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER
27        27     1     1          130  3869211170        3563
28        28     1     1          130  3869211171        3563
29        29     1     1          130  3869211171        3563


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [29]:
df_ORBIT = df.groupby('FPGA').filter(lambda x: x.sum() > 0)
print(df_ORBIT)

TypeError: filter function returned a Series, but expected a scalar bool

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel